In [1]:
import json
import pickle
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [2]:
with open('D:\\AI_Diploma\\AI_diploma\\flask\\project\\data.json', 'r') as f:
    data = json.load(f)

In [4]:
# text = []
# labels = []
# for item in data ["data"]:
#     for pattern in item["patterns"]:
#         text.append(pattern)
#         labels.append(item["label"])


text = []
labels = []

for item in data:  
    for pattern in item["patterns"]:
        text.append(pattern)
        labels.append(item["tag"])  


In [5]:
vectorizer = TfidfVectorizer(max_features=2000)
X = vectorizer.fit_transform(text).toarray()

le = LabelEncoder()
y = le.fit_transform(labels)

In [6]:
pickle.dump(vectorizer, open('vectorizer.pkl', 'wb'))
pickle.dump(le, open('label_encoder.pkl', 'wb'))

In [7]:
class chatDataset(Dataset):
    def __init__ (self,X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [8]:
dataset = chatDataset(X, y)
loader = DataLoader(dataset, batch_size=8, shuffle=True)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "d:\AI_Diploma\AI_diploma\venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "d:\AI_Diploma\AI_diploma\venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "d:\AI_Diploma\AI_diploma\venv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "d:\AI_Diploma\AI_diplom

In [9]:
# ----- Model ----- 

class ChatModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ChatModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [ ]:
input_dim = X.shape[1]
hidden_dim = 128
output_dim = len(le.classes_)

model = ChatModel(input_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


for epoch in range(30):
    total_loss = 0
    for X_batch , y_batch in loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()#update weights
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

# ----- Save model -----
torch.save(model.state_dict(), "model.pth")

print("✅ Training done and model saved as model.pth")

Epoch 1, Loss: 127.3689
Epoch 2, Loss: 122.9506
Epoch 3, Loss: 115.7915
Epoch 4, Loss: 107.8384
Epoch 5, Loss: 98.3695
Epoch 6, Loss: 93.6840
Epoch 7, Loss: 86.2258
Epoch 8, Loss: 76.7911
Epoch 9, Loss: 68.8574
Epoch 10, Loss: 61.9363
Epoch 11, Loss: 55.0299
Epoch 12, Loss: 49.0805
Epoch 13, Loss: 43.9291
Epoch 14, Loss: 38.4801
Epoch 15, Loss: 34.4010
Epoch 16, Loss: 31.3779
Epoch 17, Loss: 26.0508
Epoch 18, Loss: 22.7604
Epoch 19, Loss: 20.3766
Epoch 20, Loss: 18.3682
Epoch 21, Loss: 16.3088
Epoch 22, Loss: 15.3466
Epoch 23, Loss: 13.8944
Epoch 24, Loss: 13.3256
Epoch 25, Loss: 11.8691
Epoch 26, Loss: 11.3767
Epoch 27, Loss: 10.5228
Epoch 28, Loss: 9.8391
Epoch 29, Loss: 9.5365
Epoch 30, Loss: 9.0564
✅ Training done and model saved as model.pth


: 